In [2]:
#r "nuget: Microsoft.ML"
using Microsoft.ML;
using Microsoft.ML.Data;
using System.IO;
using System.Net;

Installed package Microsoft.ML version 1.4.0

Create a class that represents our data:

In [3]:
class AbaloneData
{
    [LoadColumn(0)]
    public string Sex { get; set; }

    [LoadColumn(1)]
    public float Length { get; set; }

    [LoadColumn(2)]
    public float Diameter { get; set; }

    [LoadColumn(3)]
    public float Height { get; set; }

    [LoadColumn(4)]
    public float WholeWeight { get; set; }

    [LoadColumn(5)]
    public float ShuckedWeight { get; set; }

    [LoadColumn(6)]
    public float VisceraWeight { get; set; }

    [LoadColumn(7)]
    public float ShellWeight { get; set; }

    [LoadColumn(8)]
    public float Rings { get; set; }
}

Download the data from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone):

In [10]:
if (!File.Exists("abalone.data"))
{
    using var client = new WebClient();
    client.DownloadFile("https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data", "abalone.data");
}
File.ReadLines("abalone.data").Take(5)

index,value
0,"M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15"
1,"M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07,7"
2,"F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21,9"
3,"M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155,10"
4,"I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055,7"


Create a new MLContext:

In [5]:
var context = new MLContext();

Read the data in the file:

In [6]:
var dataView = context.Data.LoadFromTextFile<AbaloneData>("abalone.data", hasHeader: false, separatorChar: ',');

Create a One-Hot Encoder:

In [7]:
var encoder = context.Transforms.Categorical.OneHotEncoding(inputColumnName: "Sex", outputColumnName: "EncodedSex");

Create a transformer and fit it to the data:

In [8]:
var transformer = encoder.Fit(dataView);
var transformedDataView = transformer.Transform(dataView);

Create a class to hold the value and its encoding:

In [12]:
class SexEncoding
{
    public string Sex { get; set; }
    public float[] EncodedSex { get; set; }

    public override string ToString() => $"{Sex}; [{string.Join(",", EncodedSex)}]";
}

Encode the data and randomly select 10:

In [14]:
var encodedLabels = context.Data.CreateEnumerable<SexComparer>(transformedDataView, reuseRowObject: false);

var rand = new Random();

encodedLabels
    .OrderBy(_ => rand.Next())
    .Take(10)

index,Sex,EncodedSex
0,I,"[ 0, 0, 1 ]"
1,M,"[ 1, 0, 0 ]"
2,M,"[ 1, 0, 0 ]"
3,M,"[ 1, 0, 0 ]"
4,M,"[ 1, 0, 0 ]"
5,I,"[ 0, 0, 1 ]"
6,M,"[ 1, 0, 0 ]"
7,F,"[ 0, 1, 0 ]"
8,F,"[ 0, 1, 0 ]"
9,M,"[ 1, 0, 0 ]"
